In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NMT/

/content/drive/MyDrive/NMT


In [ ]:
%ls

'=1.3.0'        log                                 train.bpe.ru   vocab.py
 bpe_rules.en   opus-mt-ru-en-finetuned-ru-to-en/   train.en
 bpe_rules.ru   __pycache__/                        train.ru
 data.txt       train.bpe.en                        utils.py


In [ ]:
%%capture
!pip3 install torch>=1.3.0
!pip3 install subword-nmt &> log

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# BPE Tokenizing

In [ ]:
# Tokenizing & applying BPE rules

tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

# split and tokenize the data
with open('train.en', 'w') as f_src,  open('train.ru', 'w') as f_dst:
    for line in open('data.txt', 'r'):
        src_line, dst_line = line.strip().split('\t')
        f_src.write(tokenize(src_line) + '\n')
        f_dst.write(tokenize(dst_line) + '\n')


# build and apply bpe vocs
bpe = {}
for lang in ['en', 'ru']:
    learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)  # 1. learn_bpe rules
    bpe[lang] = BPE(open('./bpe_rules.' + lang))  # 2. create instance of BPE class
    
    with open('train.bpe.' + lang, 'w') as f_out:
        for line in open('train.' + lang):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')  # 3. apply BPE tokenization to our data        


# Building vocabularies

data_inp = np.array(open('./train.bpe.ru').read().split('\n'))
data_out = np.array(open('./train.bpe.en').read().split('\n'))

train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=3000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')            

from vocab import Vocab

inp_voc = Vocab.from_lines(train_inp) # creates an instance of Vocab class from input lines (ru (input) vocab here)
out_voc = Vocab.from_lines(train_out) # en (output) vocab

print(f'Length of input (Russian) BPE vocabulary = {len(inp_voc)}')
print(f'Length of output (English) BPE vocabulary = {len(out_voc)}')

100%|██████████| 8000/8000 [00:26<00:00, 296.81it/s]


inp: на территории обустроена бесплатная частная парковка .
out: free private parking is available on site .

inp: кроме того , в 5 минутах ходьбы работают многочисленные бары и рестораны .
out: guests can find many bars and restaurants within a 5 - minute walk .

inp: отель san mi@@ gu@@ el расположен в центре мор@@ ели@@ и , в 750 метрах от главной площади города и кафедрального собора .
out: hotel san miguel is located in central more@@ lia , 750 metres from the city ’ s main square and cathedral .

Length of input (Russian) BPE vocabulary = 8048
Length of output (English) BPE vocabulary = 7801


In [ ]:
!pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.2 MB/s eta 0:00:00
  Preparing metad

In [ ]:
import transformers

from transformers import AutoTokenizer
    
model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=False)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Helsinki-NLP--opus-mt-ru-en/snapshots/39fdcea592bdeb244fa87ce823e9f5c70d3a2bc3/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-ru-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62517,
  "decoder_vocab_size": 62518,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "L

# Data Prep

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

def parse_ruencorp(train_inp=train_inp, dev_inp=dev_inp, train_out=train_out, dev_out=dev_out, mode='train'):
    if mode == 'train':
        for src_line, dst_line in zip(train_inp, train_out):
            yield {"translation": {"ru": src_line, "en": dst_line}}
    elif mode == 'test':
        for src_line, dst_line in zip(dev_inp, dev_out):
            yield {"translation": {"ru": src_line, "en": dst_line}}

In [ ]:
train_df = pd.DataFrame(parse_ruencorp(mode='train'))
test_df = pd.DataFrame(parse_ruencorp(mode='test'))

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets = DatasetDict({'train': train_dataset, 
                            'test': test_dataset})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 47001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/47001 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 47001
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to('cuda') # 'cuda' 

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Helsinki-NLP--opus-mt-ru-en/snapshots/39fdcea592bdeb244fa87ce823e9f5c70d3a2bc3/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-ru-en",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62517,
  "decoder_vocab_size": 62518,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "L

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("model_weights_no_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

In [ ]:
%ls

'=1.3.0'        data.txt       train.bpe.en   train.ru
 bpe_rules.en   log            train.bpe.ru   vocab.py
 bpe_rules.ru   __pycache__/   train.en


In [ ]:
!du -h model_weights.pkl

du: cannot access 'model_weights.pkl': No such file or directory


In [ ]:
model.parameters

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    logging_strategy="steps",
    logging_steps=150,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    report_to="wandb",
    
)

PyTorch: setting up devices


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-18-2551aeb982e4>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Inference & BLEU computing before fine-tuning

## Translations & BLEU before fine-tuning

In [ ]:
def translate(src_text):
    translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True).to('cuda'))
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated] 

In [ ]:
# test on 3000/700 samples 
#
#

# from tqdm import tqdm

# references = []
# predictions = []

# for inp_line, out_line in tqdm(zip(dev_inp[::700], dev_out[::700])):
#     translated_inp = translate(inp_line)
#     inp = inp_line.replace('@@ ', '')
#     #trans = translated_inp[0].replace('@@ ', '') # должно быть вот так и работать должно нормально
#     trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
#     predictions.append(trans)

#     out = out_line.replace('@@ ', '')
#     references.append(out)
    
#     print(f'input line: {inp}')
#     print(f'translated line: {trans}')
#     print(f'target line: {out}')
#     print()

In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp, dev_out)):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2688it [19:13,  2.55it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2689it [19:14,  2.05it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2690it [19:14,  2.42it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced

In [ ]:
!head -n 100 all_3k_eval_translations.txt

input line: в распоряжении гостей общая кухня и общая гостиная .
translated line: a common kitchen and a common living room are available to guests.
target line: a shared equipped kitchen and a common living room are provided to guests .

input line: на территории виллы shengsi huajing находится сад и терраса .
translated line: in villa shengsi huajing is the garden and terrace.
target line: at shengsi huajing villa you will find a garden and a terrace .

input line: расстояние от отеля libuše до ближайшей станции метро kobylisy ( линия с ), от которой можно добраться до центрального железнодорожного вокзала праги и центра города , составляет 500 метров .
translated line: the distance from the hotel libuše to the nearest subway station kobylisy (line c ) from which it is possible to reach the main railway station of praga and the city centre is 500 metres.
target line: the nearest metro station at kobylisy , on line c , is set 500 metres from hotel libuše , and it offers connections to

In [ ]:
!du -h all_3k_eval_translations.txt

1.2M	all_3k_eval_translations.txt


In [ ]:
import sacrebleu

bleu_score = sacrebleu.corpus_bleu(predictions, [references])
print("BLEU score: ", bleu_score.score)

BLEU score:  11.746981346412316


# Fine-tuning 

In [ ]:
pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb

In [ ]:
wandb.init()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 47001
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14690
  Number of trainable parameters = 76147712


Step,Training Loss,Validation Loss,Bleu,Gen Len
1000,1.379900,1.226650,34.000200,27.278000
2000,1.220400,1.101541,36.113900,27.172700
3000,1.130400,1.032956,37.579200,27.360700
4000,1.030800,0.995135,37.963600,27.072000
5000,1.012700,0.964720,39.145300,27.109300
6000,0.941300,0.940124,39.412900,26.863700
7000,0.919500,0.927302,39.667900,26.833000
8000,0.921600,0.908887,40.345200,26.854300
9000,0.854200,0.898940,40.722600,26.991700
10000,0.864000,0.890871,41.098300,27.074300


Выходные данные были обрезаны до нескольких последних строк (5000).
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id":

KeyboardInterrupt: ignored

In [ ]:
trainer.log_metrics

<bound method log_metrics of <transformers.trainer_seq2seq.Seq2SeqTrainer object at 0x7f8537bb0eb0>>

## Saving fine-tuned model weights to drive.google

In [ ]:
torch.save(model.state_dict(), 'fine-tuned-model.pt')

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("transformer_weights_after_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

## Fine-tuned transformer translations

In [ ]:
# test on 3000/700 samples 
#
#

from tqdm import tqdm

references = []
predictions = []

for inp_line, out_line in tqdm(zip(dev_inp[::500], dev_out[::500])):
    translated_inp = translate(inp_line)
    inp = inp_line.replace('@@ ', '')
    #trans = translated_inp[0].replace('@@ ', '') # должно быть вот так и работать должно нормально
    trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
    predictions.append(trans)

    out = out_line.replace('@@ ', '')
    references.append(out)
    
    print(f'input line: {inp}')
    print(f'translated line: {trans}')
    print(f'target line: {out}')
    print()

0it [00:00, ?it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

1it [00:00,  6.17it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2it [00:00,  5.93it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: в распоряжении гостей общая кухня и общая гостиная .
translated line: a common kitchen and a common living room are available to guests.
target line: a shared equipped kitchen and a common living room are provided to guests .

input line: кроме того , предоставляется прокат велосипедов , услуги трансфера и бесплатная парковка .
translated line: in addition, bike rentals, transfer services and free parking are provided.
target line: bicycle rental and shuttle services are also available . the property offers free parking .



3it [00:00,  5.11it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: расстояние до города киссимми составляет 26 км .
translated line: the distance to the town of kissimmmi is 26 km.
target line: the unit is 26 km from kissimmee .



4it [00:00,  3.62it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: апартаменты в пентхаусе с общим открытым бассейном , садом , кондиционером и террасой для загара расположены в 5 минутах ходьбы от пляжа на курорте кабо - рой .
translated line: apartments in penthouse with a shared open pool, garden, air conditioning and tanning terrace are located 5 minutes from the beach at the ka resortbo roy.
target line: situated 5 minutes ' walk from the beach in cabo roig , this air - conditioned penthouse apartment features a communal outdoor pool , garden and sun terrace .



5it [00:01,  3.35it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

6it [00:01,  4.11it/s]

input line: апартаменты moscow point - loft red square находятся в москве , в 200 метрах от большого театра .
translated line: apartments moscow point - loft red square are located in moscow, 200 metres from the big theater.
target line: moscow point - loft red square offers accommodation in moscow . the apartment is 200 metres from bolshoi theatre .

input line: в вашем распоряжении собственная ванная комната с душем и полотенцами .
translated line: you have your own bathroom with showers and towels.
target line: featuring a shower , private bathrooms also come with towels .



In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('fine-tuned-all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp, dev_out)):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2688it [17:18,  2.73it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2689it [17:19,  1.77it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2690it [17:20,  2.02it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced

# Helpful funtions

save model weights as pickle file (and upload it to your disk)

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("model_weights_no_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

load .pickle weights from disk.google

In [ ]:

# Load the saved weights
with open("model_weights.pkl", "rb") as f:
    saved_weights = pickle.load(f)

# Set the model weights to saved weights
model.load_state_dict(saved_weights)

# Use the loaded model for inference

writing translations into file (in a standartized way)